In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# standardized dataframes
data_path='/home/yuge/feature-attribution-sc'

task1_df = pd.read_csv(f'{data_path}/outputs/baselines/task1_random.csv') # this contains control
task2_df = pd.read_csv(f'{data_path}/outputs/baselines/task2_random.csv') # this contains unlabled

In [3]:
def make_features_unique(df):
    """De-duplicates features in `df` in the same way as `anndata`."""
    import anndata as ad
    dfa = ad.AnnData(df.set_index('gene_symbols').T)
    dfa.var_names_make_unique()
    df['gene_symbols'] = dfa.var_names


catch = {}
for i, file in enumerate(os.walk('../outputs/')):
    if i == 0:
        continue

    for attrib in file[2]:
        corrected = False
        path = file[0] + '/' + attrib
        
        if 'gini' in path:  # ignore gini calculations
            continue
        
        df = pd.read_csv(path)
        
        ### check that features are labeled ###
        if df.columns[0] != 'gene_symbols':
            # correct if it's just misnamed
            if type(df[[df.columns[0]]].values[0][0]) is str:  # assume genes if str
                df.columns = ['gene_symbols'] + list(df.columns[1:])
                corrected = True
    
            else:
                raise ValueError('Please provide a column named `gene_symbols` containing gene names.')

        ### check for unique features and attempt to fix ###
        if len(set(df['gene_symbols'])) != df.shape[0]:
            make_features_unique(df)
            corrected = True
                
        ### check that columns are labeled, number of columns, and values of features ###
        if 'task1.5' in attrib:
            # check columns
            
            # check features
            print('replogle')
            pass
        
        elif 'task1' in attrib:
            # checking columns
            diff = set(df.columns[1:]) - set(task1_df.columns[1:])
            if abs(len(diff)) > 0:
                
                # no problem if output was for latent dimensions
                if 'latent' in attrib:
                    pass
                
                else:
                    catch[path] = f'Categories are not the same. Saw \n{diff}'
                
            # checking features
            diff = set(df['gene_symbols'].values) - set(task1_df['gene_symbols'].values)
            if len(diff) > 0:
                catch[path] = f'Contains genes not found in the original adata!! {diff}'
            
        elif 'task2' in attrib:
            # checking columns - should be at least a subset of all `scanvi_label`
            diff = set(df.columns[1:]) - set(task2_df.columns[1:])
            if abs(len(diff)) > 0:
                catch[path] = f'Categories are not the same. Expected only {task2_df.columns[1:]} but saw \n{diff}'
               
            # checking features
            if set(df['gene_symbols'].values) != set(task2_df['gene_symbols'].values):
                catch[path] = 'Gene symbols do not match!!'
        
        elif 'task3' in attrib:
            pass
            
        else:
            raise ValueError(f'No task identified in {attrib}!!')
            
        # save auto-fixes
        if corrected:
            df.set_index('gene_symbols').to_csv(path)

for path, error in catch.items():
    print(path, error)
    display(pd.read_csv(path).head())

replogle
../outputs/integrated_gradients/task1_absolute_integrated_grads_v2.csv Contains genes not found in the original adata!! {'BCL2L11-1', 'ETS2-1', 'TSC22D1-1', 'CNN1-1', 'NCL-1', 'PRDM1-1', 'IGDCC3-1', 'COL2A1-1', 'CDKN1C-1', 'CITED1-1', 'CDKN1A-1', 'OSR2-1', 'CKS1B-1', 'CEBPE-1', 'LYL1-1', 'POU3F2-1', 'SGK1-1', 'DUSP9-1', 'ARRDC3-1', 'MIDN-1', 'HOXC13-1', 'IRF1-1', 'DLX2-1', 'ZC3HAV1-1', 'HES7-1', 'COL1A1-1', 'SPI1-1', 'CLDN6-1', 'FEV-1', 'CSRNP1-1', 'LHX1-1', 'CEBPA-1', 'KLF1-1', 'SLC4A1-1', 'IER5L-1', 'MAP7D1-1', 'KIF2C-1', 'JUN-1', 'EGR1-1'}


,gene_symbols,AHR,ARID1A,ARRDC3,ATL1,BAK1,BCL2L11,BCORL1,BPGM,C3orf72,...,TP73,TSC22D1,UBASH3A,UBASH3B,ZBTB1,ZBTB10,ZBTB25,ZC3HAV1,ZNF318,control
0,SAMD11,3.791419,2.350351,2.245172,1.287139,7.569789,2.619869,1.103736,3.203478,2.099038,...,1.497110,1.752076,3.485513,8.223483,1.785609,3.837095,1.827373,0.949604,1.583163,48.381454
1,HES4,125.529590,35.119663,109.442990,73.323660,326.370880,129.244220,89.700970,99.527580,48.091530,...,25.690529,123.084640,185.292070,301.745820,156.976000,28.369438,140.568310,100.184525,169.196210,2646.595500
2,ISG15,232.120350,77.602135,195.530150,120.136410,560.213130,222.854350,189.133180,165.868760,121.523530,...,133.315690,196.448300,310.351930,425.810850,254.518740,70.688490,265.759550,181.268400,318.567570,4690.142000
3,MMP23B,11.351492,8.222809,5.277307,7.283207,17.767334,5.726194,3.035446,6.891820,1.484689,...,13.598379,4.908533,10.795803,17.598461,18.523163,3.516813,11.059156,4.693916,8.884116,108.023360
4,SMIM1,126.898940,83.623610,93.947700,206.621660,228.456220,84.443016,83.498070,105.943800,93.159134,...,341.069950,79.637600,187.950470,311.223570,273.563020,53.552070,196.849620,61.096060,117.994120,2012.891000


../outputs/expected_gradients/task1_absolute_expected_grads_v2.csv Contains genes not found in the original adata!! {'BCL2L11-1', 'ETS2-1', 'TSC22D1-1', 'CNN1-1', 'NCL-1', 'PRDM1-1', 'IGDCC3-1', 'COL2A1-1', 'CDKN1C-1', 'CITED1-1', 'CDKN1A-1', 'OSR2-1', 'CKS1B-1', 'CEBPE-1', 'LYL1-1', 'POU3F2-1', 'SGK1-1', 'DUSP9-1', 'ARRDC3-1', 'MIDN-1', 'HOXC13-1', 'IRF1-1', 'DLX2-1', 'ZC3HAV1-1', 'HES7-1', 'COL1A1-1', 'SPI1-1', 'CLDN6-1', 'FEV-1', 'CSRNP1-1', 'LHX1-1', 'CEBPA-1', 'KLF1-1', 'SLC4A1-1', 'IER5L-1', 'MAP7D1-1', 'KIF2C-1', 'JUN-1', 'EGR1-1'}


,gene_symbols,AHR,ARID1A,ARRDC3,ATL1,BAK1,BCL2L11,BCORL1,BPGM,C3orf72,...,TP73,TSC22D1,UBASH3A,UBASH3B,ZBTB1,ZBTB10,ZBTB25,ZC3HAV1,ZNF318,control
0,SAMD11,9.368401,3.663634,5.313603,5.374637,14.862318,6.470471,2.386346,3.330462,3.113841,...,1.941336,2.340006,7.110925,15.370707,5.344365,3.606552,4.672394,1.693644,4.257837,117.71924
1,HES4,148.043550,54.633656,113.973946,84.150986,341.408330,134.469760,117.671170,105.264650,62.759598,...,53.077415,111.980750,206.069920,302.709100,171.066020,34.032080,170.209730,112.258660,157.813840,2852.89450
2,ISG15,279.444520,95.562485,206.807700,157.929780,628.249150,235.991130,222.966060,192.071870,139.942440,...,135.562590,184.415710,380.226620,531.059300,303.064400,92.108284,313.887630,212.834260,285.167020,5201.59670
3,MMP23B,17.312895,9.630321,8.677674,10.350353,33.495747,11.373658,10.231623,12.245745,7.795117,...,12.882106,8.503679,18.174097,31.108545,25.483719,4.970862,15.228096,10.479915,14.252667,239.11350
4,SMIM1,176.367780,84.738495,132.623930,182.332080,348.141500,137.483870,119.534330,134.523710,105.913270,...,245.410190,103.372420,231.701720,372.179700,257.358900,70.844160,216.640660,107.063630,151.396180,2928.95830
